In [1]:
username = "Alex"
pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")
include(pathtorepo * "dynamical-systems\\system.jl")


using StaticArrays, DifferentialEquations, DynamicalSystems, JLD
E, x, y  = -40..40, -5..5,  -5..5
box = E × x × y
using GLMakie
using LinearAlgebra, LinearSolve,Statistics

cd("C:\\Users\\" *username *  "\\Desktop\\dynamical-systems\\Tsodyks Markram\\Levanova\\3 набор параметров\\Сопоставление с матконт\\файлы matlab")
I0_hom = load("I0_hom_hom.jld")["data"]
u0_hom = load("U0_hom_hom.jld")["data"]
I0_hom = I0_hom[:]
u0_hom = u0_hom[:];

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`


In [2]:
function condition(u, t, integrator)
    x = Vector(u)
    b = x - fp[1]
    b = Vector(b)
    linprob = LinearProblem(A, b)
    linsolve = solve(linprob)
    return norm(linsolve.u, Inf) - ϵ_box
end
affect!(integrator) = terminate!(integrator)
function condition(u)
    x = Vector(u)
    b = x - fp[1]
    b = Vector(b)
    linprob = LinearProblem(A, b)
    linsolve = solve(linprob)
    return norm(linsolve.u, Inf), linsolve
end
function color_alpha(alpha)
    if alpha > 0.0
        return "red"
    else
        return "blue"
    end
end;

In [3]:
const τ = 0.013;  const τD = 0.07993;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4; const α = 1.58; const ΔU0 = 0.305

t = 100.0; tspan = [0.0, t];

In [4]:
# index point from homoclinic curve
index_hom = 1;

I0 = -1.6; U0 = u0_hom[index_hom]

p = SA[α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]
u0 = [11.325905642223786, 0.6594706953104683, 0.4863175026548461]

ds = CoupledODEs(TM, u0, p)
fp, _, _ = fixedpoints(ds, box, jacob_TM_)

ϵ_box = 1e-9
Ju0 = jacob_TM_(fp[1], p, 0);
fp_ = fp[1]

eigen_val_vec = eigen(Ju0)
eigen_vectors = eigen_val_vec.vectors

v1 = real(eigen_vectors[:, 1])
v2 = real(eigen_vectors[:, 2])
v3 = imag(eigen_vectors[:, 3])

v1 = normalize(v1, 2)
v2 = normalize(v2, 2)
v3 = normalize(v3, 2)

v1t = transpose(v1)
v2t = transpose(v2)
v3t = transpose(v3)

A = transpose([v1t; v2t; v3t])
A = Matrix(A)

N = 501
nrange = range(1, N - 1, step = 1)
dots = zeros(length(nrange), 3)
data = zeros(length(nrange), 7)
colors = []
check_events = []
time_events = []

for (i, n) in enumerate(nrange)
    α1 = 0
    α2 = cos(2 * pi * n / N) * (ϵ_box / 2)
    α3 = sin(2 * pi * n / N) * (ϵ_box / 2)
    αs = [α1, α2, α3]
    dot = fp_ + A * αs
    dots[i, :] = dot
end

In [5]:
cb = ContinuousCallback(condition, nothing, affect!)

for index in range(1, length(nrange), step = 1)
    
    u0 = SA[dots[index, 1], dots[index, 2], dots[index, 3]]
    prob = ODEProblem(TM, u0, tspan, p)
    sol = solve(prob, alg = Vern9(), abstol = 1e-14, reltol = 1e-14, callback = cb)
    norm, linsolve = condition(sol[end])

    data[index, 1] = sol[1, end]
    data[index, 2] = sol[2, end]
    data[index, 3] = sol[3, end]
    data[index, 4:6] = linsolve
    data[index, 7] = norm
    push!(colors, color_alpha(linsolve[1]))

    if sol.retcode == ReturnCode.Terminated
        push!(check_events, true)
        push!(time_events, sol.t[end])
    else
       push!(check_events, false)
    end
end

In [6]:
println("min: $(minimum(time_events))")
println("max: $(maximum(time_events))")
println("mean: $(mean(time_events))") 

min: 0.12863585849329076
max: 0.8751748911178187


mean: 0.2582576591461003


In [7]:
number_of_event = length(findall(check_events .== true))
number_of_no_event = length(check_events) - number_of_event
println("number of event: $(number_of_event); number of no event: $(number_of_no_event)")

number of event: 343; number of no event: 157


In [8]:
# last points in space (α2, α3)
f = Figure(resolution = (700, 700))

ax = Axis(f[1, 1], xlabel = L"α_2", ylabel = L"α_3", xlabelsize = 35, ylabelsize = 35)
idx, idy = 5, 6

scatter!(ax, 0.0, 0.0, color = :green, markersize = 15, marker = :rect)

scatter!(ax, data[:, idx], data[:, idy], color = colors[:], markersize = 5)


display(GLMakie.Screen(), f);

In [19]:
minimum(data[:, 6]), maximum(data[:, 6])

(-4.425618752775773, 6.686327213224743)

In [20]:
minimum(data[:, 5]), maximum(data[:, 5]) 

(-4.425618752775773, 6.686327213224743)

In [9]:
# last points in phase space
idx, idy, idz = 2, 3, 1

f = Figure(resolution = (700, 700))

ax = LScene(f[1, 1], show_axis = true)
    
scale!(ax.scene, 50, 50, 1)

scatter!(data[:, idx], data[:, idy], data[:, idz], markersize = 4.0, color = :deeppink)

scatter!(ax, fp[1][idx], fp[1][idy], fp[1][idz], color = :blue, markersize = 4.0)

display(GLMakie.Screen(), f);